![Hand with calculator](calculator.png "Calculator")

Did you know that the average return from investing in stocks is 10% per year! But who wants to be average?! 

You have been asked to support an investment firm by analyzing trends in high-growth companies. They are interested in understanding which industries are producing the highest valuations and the rate at which new high-value companies are emerging. Providing them with this information gives them a competitive insight as to industry trends and how they should structure their portfolio looking forward.

You have been given access to their `unicorns` database, which contains the following tables:

`dates`
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| company_id   | A unique ID for the company.                 |
| date_joined  | The date that the company became a unicorn.  |
| year_founded | The year that the company was founded.       |

`funding`
| Column           | Description                                  |
|----------------- |--------------------------------------------- |
| company_id       | A unique ID for the company.                 |
| valuation        | Company value in US dollars.                 |
| funding          | The amount of funding raised in US dollars.  |
| select_investors | A list of key investors in the company.      |

`industries`
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| company_id   | A unique ID for the company.                 |
| industry     | The industry that the company operates in.   |

`companies`
| Column       | Description                                       |
|------------- |-------------------------------------------------- |
| company_id   | A unique ID for the company.                      |
| company      | The name of the company.                          |
| city         | The city where the company is headquartered.      |
| country      | The country where the company is headquartered.   |
| continent    | The continent where the company is headquartered. |


### Task 1: Identify the three-best performing industries based on the number of new unicorns created between 2019, 2020 and 2021

In [12]:
WITH cte_unicorns AS (
    SELECT
        d.company_id,
        date_joined,
        DATE_PART('year', date_joined) year_created,
        industry
    FROM dates as d
    JOIN industries as i
    ON d.company_id = i.company_id
    WHERE DATE_PART('year', date_joined) BETWEEN 2019 and 2021
)
SELECT
	industry,
	count(company_id) AS num_unicorns
FROM cte_unicorns
GROUP BY industry
ORDER BY num_unicorns DESC
LIMIT 3;

,industry,num_unicorns
0,Fintech,173
1,Internet software & services,152
2,E-commerce & direct-to-consumer,75


### Task 2: Return the industry, year, number of companies in these industries that became unicorns each in 2019, 2020 and 2021, and the average valuation per industry per year in billions. Order the results by industry, then year in descending order

In [26]:
with cte_valuations AS (
    SELECT
        i.company_id,
        industry,
        DATE_PART('year', date_joined) year_created,
        (valuation / 1000000000) valuation_billions
    FROM industries as i
    JOIN dates as d
        ON i.company_id = d.company_id
    JOIN funding as f
        ON i.company_id = f.company_id
    WHERE
    	industry IN ('Fintech', 'Internet software & services', 'E-commerce & direct-to-consumer')
    	AND DATE_PART('year', date_joined) BETWEEN 2019 and 2021
)
SELECT
	industry,
    year_created AS year,
    COUNT(company_id) num_unicorns,
    ROUND(AVG(valuation_billions), 2) as average_valuation_billions
FROM cte_valuations
GROUP BY industry, year_created
ORDER BY industry, year_created DESC;

,industry,year,num_unicorns,average_valuation_billions
0,E-commerce & direct-to-consumer,2021,47,2.47
1,E-commerce & direct-to-consumer,2020,16,4.00
2,E-commerce & direct-to-consumer,2019,12,2.58
3,Fintech,2021,138,2.75
4,Fintech,2020,15,4.33
5,Fintech,2019,20,6.80
6,Internet software & services,2021,119,2.15
7,Internet software & services,2020,20,4.35
8,Internet software & services,2019,13,4.23
